# Fit with errors (weighted fit)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

True function (just example): 
$$
y(x) = \frac{a}{(b-x)^2 + c}
$$

In [ ]:
def ytrue(x):
    return 137.036 / ((4.05 - x) ** 2 + 1.0)

### Data, with noise and error bars

In [ ]:
filename = "data_with_errors.csv"
data = np.genfromtxt(filename, delimiter=",", skip_header=1)
xdata = data[:, 0]
ydata = data[:, 1]
yerror = data[:, 2]

In [ ]:
# Create denser x-grid for plotting
x = np.linspace(min(xdata), max(xdata), 100)

plt.errorbar(xdata, ydata, yerr=yerror, fmt=".", capsize=3, label="data")
plt.plot(x, ytrue(x), "k--", label="actual")
plt.legend()
plt.show()

### General fit:
* Define fitting function (model)
* Perform the fit
* Note: for complicated functions like this, we often need initial guess of parameters

In [ ]:
def f_fit(x, a, b, c):
    return a / ((b - x) ** 2 + c)

In [ ]:
# perform the fit:
initial_guess = [100, 1, 1]
popt, pcov = curve_fit(f_fit, xdata, ydata, initial_guess)

# extract the parameters from 'popt = optimised paramters' (careful of the order)
a, b, c = popt

# Extract the _approximate_ (1 sigma = standard) uncertainties from pcov (parameter covariance)
da, db, dc = np.sqrt(np.diag(pcov))

print(a, da)
print(b, db)
print(c, dc)

best = f_fit(x, a, b, c)


plt.title("Simple fit")
plt.errorbar(xdata, ydata, yerr=yerror, fmt="b.", capsize=3)
plt.plot(x, best, "r-", label="best fit")
plt.plot(x, ytrue(x), "k--", label="actual")
plt.xlabel("x")
plt.ylabel("y(x)")
plt.legend()
plt.show()

In [ ]:
residuals = f_fit(xdata, a, b, c) - ydata
plt.title("Residuals")
plt.errorbar(xdata, residuals, yerr=yerror, fmt="b.", capsize=3)
plt.show()

### Weighted fit: 
* Give curve_fit the error bars
* Weights sum of residuals

In [ ]:
# perform the fit:
popt_w, pcov_w = curve_fit(f_fit, xdata, ydata, initial_guess, yerror)

# extract the parameters from 'popt = optimised paramters' (careful of the order)
a, b, c = popt_w

# Extract the _approximate_ (1 sigma = standard) uncertainties from pcov (parameter covariance)
da, db, dc = np.sqrt(np.diag(pcov_w))

print(a, da)
print(b, db)
print(c, dc)

best = f_fit(x, a, b, c)

plt.title("Weighted fit")
plt.errorbar(xdata, ydata, yerr=yerror, fmt="b.", capsize=3)
plt.plot(x, best, "r-", label="best fit")
plt.plot(x, ytrue(x), "k--", label="actual")
plt.xlabel("x")
plt.ylabel("y(x)")
plt.legend()
plt.show()

### Gives us much better extraction of parameters:

In [ ]:
# extract the parameters from 'popt = optimised paramters' (careful of the order)
a0, b0, c0 = popt
da0, db0, dc0 = np.sqrt(np.diag(pcov))

a1, b1, c1 = popt_w
da1, db1, dc1 = np.sqrt(np.diag(pcov_w))

fig, axs = plt.subplots(ncols=3)
fig.tight_layout(pad=2.0)  # add some space

axs[0].set_title("a")
axs[0].errorbar(0.3, a0, yerr=da0, fmt="o", color="k", capsize=10)
axs[0].errorbar(0.7, a1, yerr=da1, fmt="o", color="b", capsize=10)
axs[0].set_xlim(0, 1)
axs[0].axhline(y=137.036, color="r", linestyle="-")

axs[1].set_title("b")
axs[1].errorbar(0.3, b0, yerr=db0, fmt="o", color="k", capsize=10)
axs[1].errorbar(0.7, b1, yerr=db1, fmt="o", color="b", capsize=10)
axs[1].set_xlim(0, 1)
axs[1].axhline(y=4.05, color="r", linestyle="-")

axs[2].set_title("c")
axs[2].errorbar(0.3, c0, yerr=dc0, fmt="o", color="k", capsize=10, label="simple fit")
axs[2].errorbar(0.7, c1, yerr=dc1, fmt="o", color="b", capsize=10, label="weigthed fit")
axs[2].set_xlim(0, 1)
axs[2].axhline(y=1.0, color="r", linestyle="-", label="True value")

for ax in axs:
    ax.set_xticks([])

fig.legend(loc="lower center", ncol=3)

plt.show()